In [ ]:
# 装饰器是可调用的对象其参数是另一个函数（被装饰的函数）。装饰器会处理被装饰的函数后返回，或者替换为另一个函数或可调用对象
def deco(func):
    print('running deco')
    def inner():
        print('running inner')
        return func()
    return inner

@deco
def target():
    print('running target')
print('-----------------')
target()
print(target)

In [ ]:
# 装饰器的一个关键特性是，它们在被装饰的函数定义之后立即执行，通常是导入时（即加载模块时）
registry = []  # <1>

# 将函数增加到registry中
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

# 在函数定义完成后，执行register
@register
def f1():
    print('running f1()')
print('-----------------')

# 在函数定义完成后，执行register
@register
def f2():
    print('running f2()')
print('-----------------')

# 普通函数，不会执行register
def f3():  # <7>
    print('running f3()')

def main(): 
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
print('-----------------')
print(registry)
# 依次执行三个函数，执行时
main()

# 利用装饰器改进C6中的策略模式，与前面的实现方式相比：
# - 促销策略无需使用特殊名称
# - @promotion装饰器突出了被装饰的函数的作用，还便于临时禁用某个促销策略（注释掉装饰器）
# - 促销策略可以在其他模块中定义，只要使用@promotion装饰即可

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# BEGIN STRATEGY_BEST4

promos = []  # <1>

def promotion(promo_func):  # <2>
    promos.append(promo_func)
    return promo_func

@promotion  # <3>
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):  # <4>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

# --- test
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]
long_order = [LineItem(str(item_code), 1, 1.0)
               for item_code in range(10)]

print(Order(joe, cart, best_promo))
print(Order(ann, cart, best_promo))
print(Order(joe, banana_cart, best_promo))
print(Order(joe, long_order, best_promo))

In [ ]:
# Python 不要求声明变量，但是假定在函数定义中赋值的变量时局部变量

b = 6

def f3(a):
    global b   # b被显式声明为全局变量
    print(a)
    print(b)
    b = 9

def f2(a):
    print(a)
    print(b)  # 因为有下面一句，b被作为局部变量
    b = 9

# 对比字节码，局部变量使用的是LOAD_FAST，全局变量使用的是LOAD_GLOBAL
from dis import dis
dis(f3)
print('----------')
dis(f2)
print('----------') 
f3(3)
print('----------')
f2(3)

In [ ]:
# 闭包指延伸了作用域的函数，其中包含函数定义体中引用、但是不在定义体中定义的非全局变量。
# 函数是不是匿名的没有关系，关键是它能访问定义体之外定义的非全局变量。

# 只有嵌套在其他函数中的函数才可能需要处理不在全局作用域中的外部变量
# 这类变量被称为自由变量freevars

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

avg = make_averager()
print(avg(10), avg(11), avg(12))
print(avg.__code__.co_varnames) # 局部变量
print(avg.__code__.co_freevars) # 自由变量
print(avg.__closure__) # cell对象对应自由变量
print(avg.__closure__[0].cell_contents) # 自由变量真实的值


In [ ]:
# 使用nonlocal声明，减少计算
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count

    return averager

avg = make_averager()
print(avg(10), avg(11), avg(12))

# 不使用nonlocal，count+=1 等同于 count=count+1，赋值会导致count被认为是局部变量，产生错误
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total/count

    return averager

avg = make_averager()
print(avg(10), avg(11), avg(12))

In [ ]:
# 利用装饰器，输出函数运行时间

# 实现缺点：
# - 不支持关键字参数
# - 遮盖了被装饰函数的__name__和__doc__属性。

import time


def clock(func):
    ''' clock doc '''
    def clocked(*args):
        ''' clocked doc '''
        t0 = time.time()
        result = func(*args)  # func是自由变量
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    ''' factorial doc '''
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

print(factorial.__name__, factorial.__doc__)

print('---------------')

# 使用标准库functools.wraps
import functools

def clock(func):
    ''' clock doc '''
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        ''' clocked doc '''
        t0 = time.time()
        result = func(*args, **kwargs)  # func是自由变量
        elapsed = time.time() - t0
        name = func.__name__
        arg_list = []
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    ''' factorial doc '''
    return 1 if n < 2 else n*factorial(n-1)


print('*' * 40, 'Calling snooze(.123)')
snooze(.123)
print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

print(factorial.__name__, factorial.__doc__)


In [ ]:
# functools.lru_cache

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))
print('-----------------')

# 利用lru_cache缓存中间结果，不再重复计算已经计算过的值。
# 使用字典存储结果，键值根据调用时传入的定位参数和关键字参数创建，所以被装饰函数的所有参数都必须是可散列的。
@functools.lru_cache() # 如同调用函数，接受配置参数 functools.lru_cache(maxsize=128, typed=False)
@clock  # 叠放装饰器，将@lru_cache()应用到@clock返回的函数上
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
print(fibonacci(6))

In [ ]:
# Python不支持重载方法或函数，可以通过装饰器将普通函数变成范函数generic function：
# 根据第一个参数的类型，以不同方式执行相同操作的同一组函数。
# functools.singledispatch 的显著特征：可以在系统的任何地方和模块中注册专门函数。

from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # 基函数
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{0}</pre>'.format(content)

@htmlize.register(str)  # 字符串类型
def _(text):            # _名字无关
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # int的虚拟超类
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # 叠放装饰器，同一函数支持不同类型
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))


In [ ]:
# 参数化的装饰器，创建带参数的装饰器工厂函数，在其中生成真正的装饰器

registry = set()  # <1>

def register(active=True):  # 带参数的装饰器工厂函数
    def decorate(func):  # 真正的装饰器，参数是被装饰函数
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active: 
            registry.add(func)
        else:
            registry.discard(func)

        return func  # 装饰器返回函数
    return decorate  # 工厂方法返回装饰器

@register(active=False)  # 工厂方法被调用
def f1():
    print('running f1()')
    
print('--------------')
@register()  # 工厂方法使用缺省值调用
def f2():
    print('running f2()')
print('--------------')

def f3():
    print('running f3()')
print('--------------')

print(registry)
f1()
f2()
f3()
print('--------------')
register()(f3)
print(registry)
f3()

In [ ]:
# 参数化装饰器示例

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # 参数化装饰器工厂
    def decorate(func):      # 装饰器
        def clocked(*_args): # 被装饰后的函数， *_args是clocked函数的参数
            t0 = time.time()
            _result = func(*_args)  # 调用被装饰的函数
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # 转化为字符串用于显示
            print(fmt.format(**locals()))  # **locals 是为了使用fmt中的局部变量
            return _result  # 不改变被装饰的函数
        return clocked  # 装饰器返回装饰后的函数
    return decorate  # 工厂返回装饰器

@clock()  # 缺省方式
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

clock('{name}: {elapsed}')(time.sleep)(.2)
clock('{name}({args}) dt={elapsed:0.3f}s')(time.sleep)(.2)